# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

### Mariusz Kuchta
gr. 2a - czwartek 10.00 - 11.30

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź liczbę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [1]:
import numpy as np

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');

1. Znajdź liczbę wszystkich filmów o tej samej długości.

In [2]:
req = f"""select distinct f.length as lenght, count(f.length) as films_no
                    from film f
                    group by f.length
                    having count(f.length) > 1"""

res = pd.read_sql_query(req, con=connection)

res

,lenght,films_no
0,171,8
1,177,6
2,141,7
3,127,5
4,139,11
...,...,...
135,106,6
136,61,10
137,144,8
138,178,10


In [3]:
res['films_no'].sum()

1000

2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [4]:
req = f"""select distinct ct.city as city, count(ct.city) 
                    from customer cs
                    join address ad on ad.address_id = cs.address_id
                    join city ct on ct.city_id = ad.city_id
                    group by ct.city
                    order by count asc"""

pd.read_sql_query(req, con=connection)

,city,count
0,A Corua (La Corua),1
1,Abha,1
2,Abu Dhabi,1
3,Acua,1
4,Adana,1
...,...,...
592,Zhezqazghan,1
593,Zhoushan,1
594,Ziguinchor,1
595,Aurora,2


In [5]:
req = f"""select cs.*
                    from customer cs
                    join address ad on ad.address_id = cs.address_id
                    join city ct on ct.city_id = ad.city_id
                    group by cs.customer_id"""

pd.read_sql_query(req, con=connection)


,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active
0,251,2,Vickie,Brewer,vickie.brewer@sakilacustomer.org,255,True,2006-02-14,2013-05-26 14:49:45.738,1
1,106,1,Connie,Wallace,connie.wallace@sakilacustomer.org,110,True,2006-02-14,2013-05-26 14:49:45.738,1
2,120,2,Sylvia,Ortiz,sylvia.ortiz@sakilacustomer.org,124,True,2006-02-14,2013-05-26 14:49:45.738,1
3,285,1,Miriam,Mckinney,miriam.mckinney@sakilacustomer.org,290,True,2006-02-14,2013-05-26 14:49:45.738,1
4,264,1,Gwendolyn,May,gwendolyn.may@sakilacustomer.org,269,True,2006-02-14,2013-05-26 14:49:45.738,1
...,...,...,...,...,...,...,...,...,...,...
594,328,2,Jeffrey,Spear,jeffrey.spear@sakilacustomer.org,333,True,2006-02-14,2013-05-26 14:49:45.738,1
595,263,1,Hilda,Hopkins,hilda.hopkins@sakilacustomer.org,268,True,2006-02-14,2013-05-26 14:49:45.738,1
596,144,1,Clara,Shaw,clara.shaw@sakilacustomer.org,148,True,2006-02-14,2013-05-26 14:49:45.738,1
597,7,1,Maria,Miller,maria.miller@sakilacustomer.org,11,True,2006-02-14,2013-05-26 14:49:45.738,1


3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [6]:
req = f"""
    select AVG(amount)
    from payment 
"""

pd.read_sql_query(req, con=connection)

,avg
0,4.200606


4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [7]:
req = f"""select distinct cat.name as category, count(cat.name) 
                    from film f  
                    join film_category fc on fc.film_id = f.film_id
                    join category cat on cat.category_id = fc.category_id  
                    group by cat.name"""

res = pd.read_sql_query(req, con=connection)

res

,category,count
0,Action,64
1,Animation,66
2,Children,60
3,Classics,57
4,Comedy,58
5,Documentary,68
6,Drama,62
7,Family,69
8,Foreign,73
9,Games,61


In [8]:
res['count'].sum()

1000

5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [9]:
req = f"""select distinct ct.country as country, count(ct.country) as customer_no
                    from customer cs  
                    join address ad on ad.address_id = cs.address_id
                    join city c on c.city_id = ad.city_id
                    join country ct on ct.country_id = c.country_id
                    group by ct.country"""

pd.read_sql_query(req, con=connection)

,country,customer_no
0,Malaysia,3
1,Sudan,2
2,"Congo, The Democratic Republic of the",2
3,Algeria,3
4,Zambia,1
...,...,...
103,Venezuela,7
104,Faroe Islands,1
105,North Korea,1
106,Switzerland,3


6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [10]:
req = f"""select str.*
            from store str
            join customer cs on cs.store_id = str.store_id
            group by str.store_id
            having count(cs.customer_id) > 100 and count(cs.customer_id) < 300
            """

pd.read_sql_query(req, con=connection)

,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [11]:
req =  f"""select cs.first_name as first_name, cs.last_name as last_name, SUM(f.length) as sum
            from film f
            join inventory i on i.film_id = f.film_id
            join rental r on r.inventory_id = i.inventory_id
            join customer cs on cs.customer_id = r.customer_id
            group by cs.first_name, cs.last_name
            having SUM(f.length) > 200
            order by sum, last_name, first_name"""

pd.read_sql_query(req, con=connection)

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


8. Oblicz średnią wartość wypożyczenia filmu.

In [12]:
req = f"""
    select AVG(amount)
    from payment 
"""

pd.read_sql_query(req, con=connection)

,avg
0,4.200606


9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [13]:
req = f"""select distinct cat.name as category, AVG(f.length) 
                    from film f  
                    join film_category fc on fc.film_id = f.film_id
                    join category cat on cat.category_id = fc.category_id  
                    group by cat.name"""

pd.read_sql_query(req, con=connection)

,category,avg
0,Action,111.609375
1,Animation,111.015152
2,Children,109.800000
3,Classics,111.666667
4,Comedy,115.827586
5,Documentary,108.750000
6,Drama,120.838710
7,Family,114.782609
8,Foreign,121.698630
9,Games,127.836066


10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [14]:
req = f"""select cat.name as category, f.title, MAX(LENGTH(f.title)) as max_title_len, LENGTH(f.title) as title_len
                    from film f
                    join film_category fc on fc.film_id = f.film_id
                    join category cat on cat.category_id = fc.category_id
                    where (LENGTH(f.title), cat.name) in
                       (SELECT MAX(LENGTH(fi.title)), cati.name
	                    from film fi
                        join film_category fci on fci.film_id = fi.film_id
                        join category cati on cati.category_id = fci.category_id
                        group by cati.name)
                    group by cat.name, f.title
                    order by cat.name asc"""

pd.read_sql_query(req, con=connection)

,category,title,max_title_len,title_len
0,Action,Entrapment Satisfaction,23,23
1,Animation,Telemark Heartbreakers,22,22
2,Children,Heartbreakers Bright,20,20
3,Children,Microcosmos Paradise,20,20
4,Children,Sweethearts Suspects,20,20
5,Classics,Extraordinary Conquerer,23,23
6,Comedy,Trainspotting Strangers,23,23
7,Documentary,Deliverance Mulholland,22,22
8,Documentary,Intolerable Intentions,22,22
9,Drama,Goldfinger Sensibility,22,22


11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [15]:
req = f"""select cat.name as category, f.title, MAX(f.length) as max_film_len, f.length as film_len
                    from film f
                    join film_category fc on fc.film_id = f.film_id
                    join category cat on cat.category_id = fc.category_id
                    where (f.length, cat.name) in
                       (SELECT MAX(fi.length), cati.name
	                    from film fi
                        join film_category fci on fci.film_id = fi.film_id
                        join category cati on cati.category_id = fci.category_id
                        group by cati.name)
                    group by cat.name, f.title, f.length
                    order by cat.name asc"""

pd.read_sql_query(req, con=connection)

,category,title,max_film_len,film_len
0,Action,Darn Forrester,185,185
1,Action,Worst Banger,185,185
2,Animation,Gangs Pride,185,185
3,Animation,Pond Seattle,185,185
4,Children,Fury Murder,178,178
5,Children,Wrong Behavior,178,178
6,Classics,Conspiracy Spirit,184,184
7,Comedy,Control Anthem,185,185
8,Documentary,Wife Turn,183,183
9,Documentary,Young Language,183,183


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [16]:
from main import *

```film_in_category(category_id:int)->pd.DataFrame```

In [17]:
film_in_category(12)

,title,languge,category
0,Alaska Phantom,English,Music
1,Alone Trip,English,Music
2,Amelie Hellfighters,English,Music
3,Balloon Homeward,English,Music
4,Banger Pinocchio,English,Music
5,Birch Antitrust,English,Music
6,Birdcage Casper,English,Music
7,Boogie Amelie,English,Music
8,Chamber Italian,English,Music
9,Clones Pinocchio,English,Music


```number_films_in_category(category_id:int)->pd.DataFrame```

In [18]:
number_films_in_category(2)

,category,count
0,Animation,66


```number_film_by_length(min_length: Union[int,float] = 0, max_length: Union[int,float] = 1e6 )```

In [19]:
number_film_by_length(80, 100.5)

,length,count
0,80,10
1,81,3
2,82,7
3,83,6
4,84,13
5,85,17
6,86,5
7,87,6
8,88,4
9,89,6


```client_from_city(city:str)->pd.DataFrame```

In [20]:
client_from_city('London')

,city,first_name,last_name
0,London,Cecil,Vines
1,London,Mattie,Hoffman


```avg_amount_by_length(length:Union[int,float])->pd.DataFrame```

In [21]:
avg_amount_by_length(80)

,length,avg
0,80,5.48359


```client_by_sum_length(sum_min:Union[int,float])->pd.DataFrame```

In [22]:
client_by_sum_length(1000)

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


```category_statistic_length(name:str)->pd.DataFrame)```

In [23]:
category_statistic_length('Sports')

,category,avg,sum,min,max
0,Sports,128.202703,9487,47,184
